In [96]:
def prime(fn):
    def f(*argv, **kwargs):
        res = fn(*argv, **kwargs)
        res.send(None)
        return res
    return f

def assertEq(a, b):
    assert a == b, "%s != %s" %( a, b)


@prime
def range():
    start, end = yield ()
    prev, = yield ('ok', )
    while start < end:
        assertEq(prev, "next")
        prev, = yield ('yield', start)
        start += 1

@prime
def double():
    (arg, ) = yield ()
    while True:
        (arg,) = yield ('ok', 2*arg)

@prime
def map():
    prev, = yield ()
    while True:
        assertEq(prev, "next")

        next_item, *args = yield ("req", "iter", "next")
        if next_item == "yield":
            val, = args
            ok, val = yield ("req", "fn", val)
            assertEq(ok, 'ok')

            prev, = yield("yield", val)
        elif next_item == "eos":
            yield ('eos',)
            raise 'dead'

@prime
def bind(f, name, bound):
    prev = yield None
    while True:
        resp, *rest = f.send(prev)
        if resp == 'req':
            target, *rest = rest
            if target == name:
                fnresp = bound.send(rest)
                prev = fnresp
            else:
                prev = yield ('req', target, *rest)
        else:
            prev = yield (resp, *rest)

r = map()
r = bind(r, 'fn', double())

rng = range()
assertEq(rng.send((5, 10)), ('ok',))

b = bind(r, 'iter', rng)

assertEq(b.send(('next',)), ('yield', 10))
assertEq(b.send(('next',)), ('yield', 12))


In [102]:
def start():
    tag, *rest = yield ('req', 'iter')
    if tag == 'eos':
        yield ('eos',)
    elif tag == 'yield':
        char, = rest
        yield ('startrow', )
        yield ('startfield', )
        yield from default_handle_char(char)


def quoted():
    tag, *rest = yield ('req', 'iter')
    if tag == 'eos':
        yield ('err', 'non-terminated quote')
        return
    elif tag == 'yield':
        char, = rest
        if char == "\"":
            yield from non_quoted()
        else:
            yield ('char', char)            
            yield from quoted()

def non_quoted():
    tag, *rest = yield ('req', 'iter')
    if tag == 'eos':
        yield ('endfield',)
        yield ('endrow',)
        yield ('eos',)
        return
    elif tag == 'yield':
        char, = rest
        yield from default_handle_char(char)
       

def default_handle_char(char):
    if char == '\n':
        yield ('endfield', )
        yield ('endrow',)
        yield from start()
    elif char == "\"":
        yield from quoted()
    elif char == ",":
        yield ('endfield', )
        yield ('startfield', )
        yield from non_quoted()
    elif char == '\\':
        yield from escaped()
    else:
        yield ('char', char)
        yield from non_quoted()

def escaped():
    tag, *rest = yield ('req', 'iter')
    if tag == 'eos':
        yield ('err', 'non-terminated escape sequence')
        return
    elif tag == 'yield':
        char, = rest
        yield ('char', char)
        yield from non_quoted()


def parse_csv():
    yield from start()

TEST_CSV = """foo,bar,,
"qux","bl,t"
far\,gl\\"e
"""

@prime
def string_iter(s):
    print('here')
    yield None
    print('going')
    for c in s:
        print(c)
        yield ('yield', c)
    print("end")
    yield ('eos', )




In [113]:

@prime
def cell():
    value = None
    tag, *rest = yield None
    while True:
        if tag == 'set':
            value, = rest
            tag, *rest = yield ('ok', )
        elif tag == 'take':
            tag, *rest = yield ('ok', value)
            value = None
        else:
            assert False, "bad tag"


@prime
def vector():
    storage = []
    tag, *rest = yield None
    while True:
        if tag == 'push':
            v, = rest
            storage.append(v())
            tag, *rest = yield ('ok', )
        elif tag == 'do':
            idx, *rest = rest
            tag, *rest = yield storage[idx].send(rest)
        else:
            assert False, "bad tag"


v = vector()

v.send(('push', vector))
v.send(('do', 0, 'push', cell))
v.send(('do', 0, 'do', 0, 'set', 'test'))
v.send(('do', 0, 'do', 0, 'take'))


def collect_csv():
    parser = parse_csv()
    tag, *rest = parser.send(None)
    rows = 0
    fields = 0
    chars = 0
    while True:
        print("AHHH", tag)
        if tag == 'startrow':
            tag, *rest = yield ('req', 'output', 'push', vector)
            rows += 1
            fields = 0
            chars = 0
        elif tag == 'startfield':
            tag, *rest = yield ('req', 'output', 'do', rows - 1, 'push', vector)
            fields += 1
            chars = 0
        elif tag == 'char':
            char, = rest
            tag, *rest = yield ('req', 'output', 'do', rows - 1, 'do', fields-1, 'push', cell)
            chars += 1
            tag, *rest = yield ('req', 'output', 'do', rows - 1, 'do', fields-1, 'do', chars - 1, 'set', char)
        else:
            print("else?!")
            x = yield (tag, *rest)
            print("yoielded?!", x)



i = iter(TEST_CSV)
p = collect_csv()

p = bind(p, 'iter', string_iter(TEST_CSV))
p.send(None)
# p.send(('yield', 'f'))
# while True:
#     print(tag, rest)
#     if tag == 'eos':
#         break
#     tag, *rest = p.send(())

here
AHHH req
else?!
going
f
yoielded?! ('yield', 'f')
AHHH req
else?!
o
yoielded?! ('yield', 'o')
AHHH req
else?!
o
yoielded?! ('yield', 'o')
AHHH req
else?!
,
yoielded?! ('yield', ',')
AHHH req
else?!
b
yoielded?! ('yield', 'b')
AHHH req
else?!
a
yoielded?! ('yield', 'a')
AHHH req
else?!
r
yoielded?! ('yield', 'r')
AHHH req
else?!
,
yoielded?! ('yield', ',')
AHHH req
else?!
,
yoielded?! ('yield', ',')
AHHH req
else?!


yoielded?! ('yield', '\n')
AHHH req
else?!
"
yoielded?! ('yield', '"')
AHHH req
else?!
q
yoielded?! ('yield', 'q')
AHHH req
else?!
u
yoielded?! ('yield', 'u')
AHHH req
else?!
x
yoielded?! ('yield', 'x')
AHHH req
else?!
"
yoielded?! ('yield', '"')
AHHH req
else?!
,
yoielded?! ('yield', ',')
AHHH req
else?!
"
yoielded?! ('yield', '"')
AHHH req
else?!
b
yoielded?! ('yield', 'b')
AHHH req
else?!
l
yoielded?! ('yield', 'l')
AHHH req
else?!
,
yoielded?! ('yield', ',')
AHHH req
else?!
t
yoielded?! ('yield', 't')
AHHH req
else?!
"
yoielded?! ('yield', '"')
AHHH req
else?!


yo

RuntimeError: generator raised StopIteration

# Attic

In [5]:

def csv_fsm():
    state = 'start'
    while True:
        tag, *rest = yield ('getchar', )
        if tag == 'eos':
            if state in ['start']:
                yield ('eos',)
                return
            elif state == 'between_fields':
                yield ('endfield',)
                yield ('endrow',)
                yield ('eos',)
                return
            elif state == 'non_quoted':
                yield ('endfield',)
                yield ('endrow',)
                yield ('eos',)
                return
            elif state == 'quoted':
                yield ('err', 'non-terminated quote')
                return
            elif state == 'escaped':
                yield ('err', 'non-terminated escape sequence')
                return
            else:
                assert False, "Bad state"
        elif tag == 'yield':
            char, = rest
            if state in ['start']:
                yield ('startrow', )
                yield ('startfield', )
                if char == '\n':
                    yield ('endfield', )
                    yield ('endrow')
                    state = 'start'
                elif char == "\"":
                    state = 'quoted'
                elif char == ",":
                    yield ('endfield', )
                    yield ('startfield', )
                    state = 'between_fields'
                elif char == '\\':
                    state = 'escaped'
                else:
                    yield ('char', char)
                    state = 'non_quoted'
            elif state == 'between_fields':
                if char == '\n':
                    yield ('endfield', )
                    yield ('endrow',)
                    state = 'start'
                elif char == "\"":
                    state = 'quoted'
                elif char == ",":
                    yield ('endfield', )
                    yield ('startfield', )
                    state = 'between_fields'
                elif char == '\\':
                    state = 'escaped'
                else:
                    yield ('char', char)
                    state = 'non_quoted'
            elif state == 'non_quoted':
                if char == '\n':
                    yield ('endfield', )
                    yield ('endrow',)
                    state = 'start'
                elif char == "\"":
                    state = 'quoted'
                elif char == ",":
                    yield ('endfield', )
                    yield ('startfield', )
                    state = 'between_fields'
                elif char == '\\':
                    state = 'escaped'
                else:
                    yield ('char', char)
            elif state == 'quoted':
                if char == "\"":
                    state = 'non_quoted'
                else:
                    yield ('char', char)
            elif state == 'escaped':
                yield ('char', char)
                state = "non_quoted"
            else:
                assert False, "Bad state"


TypeError: can't send non-None value to a just-started generator